In [4]:
import pandas as pd
import math
from tqdm import tqdm
import numpy as np
from util import assign_classification

# https://softwareprocess.es/2022/STBD.pm

source file| test file| document| build file

In [2]:
df = pd.read_csv('../660_data/cross_entropy.csv')

In [13]:
df_list = []
for i in range(4):
    df_list.append(pd.read_csv('../660_data/cross_entropy_'+str(i)+'.csv', index_col=False))
authorship = pd.concat(df_list)
authorship = authorship.rename(columns={'Unnamed: 0':'project_file'})

In [94]:
df['project_file'] = df['project_name'] + df['change_file']

/tmp/ipykernel_41105/1185604694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['project_file'] = df['project_name'] + df['change_file']


In [61]:
df['file_type'].value_counts()

O    3336485
S     480442
T      71309
D      55621
B       2264
Name: file_type, dtype: int64

In [132]:
full_df = pd.read_csv('/home/weijiesun/cmput/660_data/cross_entropy_full.csv', index_col=False)
full_df = full_df[~full_df['change_file'].isna()]
full_df['project_file'] = full_df['project_name'] + full_df['change_file']

In [116]:
print ('commit:', full_df['commit_sha'].nunique())
print ('author:', full_df['author_email'].nunique())
print ('project_name:', full_df['project_name'].nunique())

commit: 47676
author: 3990
project_name: 1846


In [120]:
temp = full_df.groupby('project_name').agg(['nunique'])

In [128]:
col_proj_names = temp[(temp['author']['nunique'] > 1) & (temp['author_email']['nunique'] > 1)].index.tolist()

In [267]:
col_df = full_df[full_df['project_name'].isin(col_proj_names)]

In [244]:
count_author_per_project = full_df.groupby('project_name')['author_email'].agg('nunique')

In [252]:
count_author_per_project[count_author_per_project == 2].index.tolist()

In [268]:
col_df['project_file'].nunique()

666449

In [269]:
col_df = col_df.drop_duplicates('project_file')

In [270]:
col_df = col_df[['project_name', 'project_file', 'change_file']].merge(authorship_df[['project_file', 'cross_entropy']], how='left', left_on='project_file', right_on='project_file')

In [271]:
# col_df
col_df['cross_entropy'] = col_df['cross_entropy'].fillna(value=min_cross_entropy)

In [272]:
col_df = assign_classification(col_df)

In [273]:
col_df[col_df['file_type'] == 'O']

,project_name,project_file,change_file,cross_entropy,file_type
1,wang70880_TSimplex,wang70880_TSimplexoperatorText.pyc,operatorText.pyc,2.000000e-11,O
3,wang70880_TSimplex,wang70880_TSimplexsimplex.pyc,simplex.pyc,2.000000e-11,O
8,wang70880_TSimplex,wang70880_TSimplexdatabase.pyc,database.pyc,2.000000e-11,O
16,antlampas_pipeline,antlampas_pipeline__pycache__/gestore_segnali....,__pycache__/gestore_segnali.cpython-37.pyc,2.000000e-11,O
17,antlampas_pipeline,antlampas_pipeline__pycache__/oggetto.cpython-...,__pycache__/oggetto.cpython-37.pyc,2.000000e-11,O
...,...,...,...,...,...
666411,1994benc_homeworkor-au,1994benc_homeworkor-auconfig/environment.produ...,config/environment.production.json,2.000000e-11,O
666413,1994benc_homeworkor-au,1994benc_homeworkor-aucypress.json,cypress.json,2.000000e-11,O
666414,1994benc_homeworkor-au,1994benc_homeworkor-audocker-compose.yml,docker-compose.yml,2.000000e-11,O
666432,1994benc_homeworkor-au,1994benc_homeworkor-autsconfig.json,tsconfig.json,2.000000e-11,O


In [220]:
# check commits of multiple authors
# col_df[]
temp_project_file = col_df[(col_df['cross_entropy'] == min_cross_entropy) &\
                           (col_df['file_type'] != 'A')]['project_file'].unique().tolist()
# full_df[full_df['project_file'].isin(temp_project_file)]['commit_sha'].nunique()
temp_numpy = full_df[full_df['project_file'].isin(temp_project_file)].groupby('project_file')['commit_sha'].agg('nunique').to_numpy()
print (np.sum(temp_numpy))
print (np.mean(temp_numpy))
print (np.median(temp_numpy))
print (np.percentile(temp_numpy, 25))
print (np.percentile(temp_numpy, 75))

778063
1.6848411223857844
1.0
1.0
2.0


In [226]:
# check commits of multiple authors
# col_df[]
temp_numpy = col_df[(col_df['cross_entropy']  min_cross_entropy) &\
                           (col_df['file_type'] == 'T')]['cross_entropy'].to_numpy()
# full_df[full_df['project_file'].isin(temp_project_file)]['commit_sha'].nunique()
# temp_numpy = full_df[full_df['project_file'].isin(temp_project_file)]
# print (np.sum(temp_numpy))
print (np.mean(temp_numpy))
print (np.median(temp_numpy))
print (np.percentile(temp_numpy, 25))
print (np.percentile(temp_numpy, 75))

0.12818268392868928
2.000000165460742e-11
2.000000165460742e-11
2.000000165460742e-11


In [145]:
min_cross_entropy = math.log(1.00000000002)
col_df['cross_entropy'] = min_cross_entropy
with tqdm(total=col_df['project_file'].nunique()) as pbar:
    for project_file in df['project_file'].unique():
        temp = authorship_df[authorship_df['project_file'] == project_file]
        if temp.shape[0] > 0:
            cross_entropy = authorship_df[authorship_df['project_file'] == project_file].iloc[0]['cross_entropy']
            col_df.loc[col_df['project_file'] == project_file, 'cross_entropy'] = cross_entropy
        pbar.update(1)

/tmp/ipykernel_41105/570546702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_df['cross_entropy'] = min_cross_entropy
  0%|                                    | 556/666449 [00:19<6:25:20, 28.80it/s]


KeyboardInterrupt: 

In [138]:
col_df.merge(authorship_df[['project_file', 'cross_entropy']], how='inner', left_on='project_file', right_on='project_file')

,project_name,author,author_timestamp,author_email,commit_sha,change_file,project_file,cross_entropy
0,antlampas_pipeline,antlampas,1592308371,francescoantonetti1@gmail.com>,44b28f2f2878297a39f000e15499487319190f33,gestore_segnali.py,antlampas_pipelinegestore_segnali.py,0.298436
1,antlampas_pipeline,antlampas,1592911062,francescoantonetti1@gmail.com>,21397492396fbb8ab52041f390a79767e02f5192,gestore_segnali.py,antlampas_pipelinegestore_segnali.py,0.298436
2,antlampas_pipeline,antlampas,1605017554,francescoantonetti1@gmail.com>,61b7a826b737a7707c46619b28478981b776591a,gestore_segnali.py,antlampas_pipelinegestore_segnali.py,0.298436
3,antlampas_pipeline,Francesco Antonetti Lamorgese Passeri,1625648412,francescoantonetti1@gmail.com>,2147b7bdaf9c3a815b52b4f8f429e9f8dafc9756,gestore_segnali.py,antlampas_pipelinegestore_segnali.py,0.298436
4,antlampas_pipeline,antlampas,1592299392,francescoantonetti1@gmail.com>,78288ac8436a7b209d902ef0d7c3c7554f95c286,gestore_segnali.py,antlampas_pipelinegestore_segnali.py,0.298436
...,...,...,...,...,...,...,...,...
2612207,1994benc_homeworkor-au,Ben Chomsang,1595164841,1994benc@gmail.com>,f985ea326324bc4c7b629de0a89409f908939927,yarn.lock,1994benc_homeworkor-auyarn.lock,0.562335
2612208,1994benc_homeworkor-au,Ben Chomsang,1595368320,1994benc@gmail.com>,25ac69925b6d2b5cbaa536dc0f595721c3eea786,yarn.lock,1994benc_homeworkor-auyarn.lock,0.562335
2612209,1994benc_homeworkor-au,dependabot[bot],1599179727,49699333+dependabot[bot]@users.noreply.github....,52d0418b555ecb2c44ed28f641fd39af355cad96,package-lock.json,1994benc_homeworkor-aupackage-lock.json,0.636514
2612210,1994benc_homeworkor-au,dependabot[bot],1599179724,49699333+dependabot[bot]@users.noreply.github....,52d0418b555ecb2c44ed28f641fd39af355cad96,package-lock.json,1994benc_homeworkor-aupackage-lock.json,0.636514


Z: CE, 
X: number authors
Y: count

In [297]:
count,division = np.histogram(temp_cross_entropy)

In [315]:
col_df

,project_name,project_file,change_file,cross_entropy,file_type
0,wang70880_TSimplex,wang70880_TSimplexoperatorText.py,operatorText.py,2.000000e-11,S
1,wang70880_TSimplex,wang70880_TSimplexoperatorText.pyc,operatorText.pyc,2.000000e-11,O
2,wang70880_TSimplex,wang70880_TSimplexsimplex.py,simplex.py,2.000000e-11,S
3,wang70880_TSimplex,wang70880_TSimplexsimplex.pyc,simplex.pyc,2.000000e-11,O
4,wang70880_TSimplex,wang70880_TSimplexstart.py,start.py,2.000000e-11,S
...,...,...,...,...,...
666444,1994benc_homeworkor-au,1994benc_homeworkor-ausrc/resources/models/sub...,src/resources/models/subject-model.ts,2.000000e-11,S
666445,1994benc_homeworkor-au,1994benc_homeworkor-ausrc/resources/models/use...,src/resources/models/user-model.ts,2.000000e-11,S
666446,1994benc_homeworkor-au,1994benc_homeworkor-ausrc/resources/value-conv...,src/resources/value-converters/blob-to-url-val...,2.000000e-11,S
666447,1994benc_homeworkor-au,1994benc_homeworkor-ausrc/resources/value-conv...,src/resources/value-converters/delta-to-html-v...,2.000000e-11,S


In [388]:
X, Y, Z = [],[],[]

for c_author in range(2,11):
    temp_project = count_author_per_project[count_author_per_project == c_author].index.tolist()
    temp_cross_entropy = col_df[(col_df['cross_entropy'] > min_cross_entropy) &\
       (col_df['project_name'].isin(temp_project)) &\
       (col_df['file_type'] == 'D')
                               ]['cross_entropy'].tolist()
    count,division = np.histogram(temp_cross_entropy)

    Z.append(count.tolist())
    Y.append(division.tolist()[:-1])
    X.append([c_author]*len(count))
    
X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)


In [5]:
df = pd.read_csv('../660_data/RQ_1.csv', index_col=False)

In [11]:
df = pd.read_csv('../660_data/cross_entropy.csv', index_col=False)
df

,project_name,author,author_email,change_file
0,wang70880_TSimplex,wang,wang@wjcuhk@gmail.com>,operatorText.py
1,wang70880_TSimplex,wang,wang@wjcuhk@gmail.com>,operatorText.pyc
2,wang70880_TSimplex,wang,wang@wjcuhk@gmail.com>,simplex.py
3,wang70880_TSimplex,wang,wang@wjcuhk@gmail.com>,simplex.pyc
4,wang70880_TSimplex,wang,wang@wjcuhk@gmail.com>,start.py
...,...,...,...,...
3946118,electrified_testing,Ed Brindley,edward.brindley@skybettingandgaming.com>,test.txt
3946119,electrified_testing,Ed Brindley,ed.brindley@gmail.com>,README
3946120,electrified_testing,Ed Brindley,ed.brindley@gmail.com>,demo/Makefile
3946121,electrified_testing,Ed Brindley,ed.brindley@gmail.com>,demo/source/demo.c


In [9]:
full_df = pd.read_csv('/home/weijiesun/cmput/660_data/cross_entropy_full.csv', index_col=False)


In [ ]:
authorship

,Unnamed: 0,cross_entropy,change_file
0,antlampas_pipelinegestore_segnali.py,0.298436,gestore_segnali.py
1,antlampas_pipelinegestore_pipeline.py,0.332594,gestore_pipeline.py
2,antlampas_pipelineoggetto.py,0.429323,oggetto.py
3,antlampas_pipelineREADME.md,0.450561,README.md
4,antlampas_pipelinelettore_codice_a_barre.conf,0.693147,lettore_codice_a_barre.conf
...,...,...,...
54642,winzeros_casinofantasmatrunk/src/control/juego...,0.636514,trunk/src/control/juegos/Garrafas/Main.java
54643,wqdjl_angular2-tsindex.html,0.500402,index.html
54644,worldrules_cvdepixelpackage-lock.json,0.693147,package-lock.json
54645,worldrules_cvdepixeldist/about.html,0.500402,dist/about.html
